In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [8]:
import torch

# Gravitational constant
G = 1.0  # Normalize for simplicity

# Masses
m1 = 1.0
m2 = 1.0

# Initial positions (2D vectors)
x1_0 = torch.tensor([1.0, 0.0], dtype=torch.float32)
x2_0 = torch.tensor([-1.0, 0.0], dtype=torch.float32)

# Initial velocities (2D vectors)
v1_0 = torch.tensor([0.0, 0.5], dtype=torch.float32)
v2_0 = torch.tensor([0.0, -0.5], dtype=torch.float32)

# Prediction time
T = 5.0  # seconds


In [9]:
from scipy.integrate import solve_ivp
import numpy as np

def two_body_ode(t, y, m1, m2):
    x1 = y[:2]
    v1 = y[2:4]
    x2 = y[4:6]
    v2 = y[6:8]
    
    r12 = np.linalg.norm(x2 - x1)
    
    a1 = G * m2 * (x2 - x1) / r12**3
    a2 = G * m1 * (x1 - x2) / r12**3
    
    return np.concatenate([v1, a1, v2, a2])

y0 = np.concatenate([x1_0, v1_0, x2_0, v2_0])
t_span = (0, T)
t_eval = np.linspace(0, T, 100)

sol = solve_ivp(two_body_ode, t_span, y0, t_eval=t_eval, args=(m1, m2))

# Ground truth trajectories
x1_true = sol.y[:2].T  # shape: (timesteps, 2)
x2_true = sol.y[4:6].T
t_data = sol.t.reshape(-1, 1)


In [10]:
import torch.nn as nn

class PINN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(1, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 8)  # Output: x1, v1, x2, v2 (each 2D)
        )

    def forward(self, t):
        return self.net(t)

model = PINN()


In [12]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

t_train = torch.tensor(t_data, dtype=torch.float32, requires_grad=True)
x1_train = torch.tensor(x1_true, dtype=torch.float32)
x2_train = torch.tensor(x2_true, dtype=torch.float32)

def ode_residual(model, t, m1, m2):
    pred = model(t)
    x1, v1, x2, v2 = torch.split(pred, 2, dim=1)

    # First derivatives (velocities): dx/dt = v, so we get dv/dt = a
    a1 = torch.zeros_like(v1)
    a2 = torch.zeros_like(v2)

    for i in range(2):  # loop over x and y components
        grad_v1_i = torch.autograd.grad(v1[:, i], t, grad_outputs=torch.ones_like(v1[:, i]), create_graph=True)[0]
        grad_v2_i = torch.autograd.grad(v2[:, i], t, grad_outputs=torch.ones_like(v2[:, i]), create_graph=True)[0]
        a1[:, i] = grad_v1_i.squeeze()
        a2[:, i] = grad_v2_i.squeeze()

    # Compute true acceleration from Newton's law
    r = x2 - x1
    r_norm = torch.norm(r, dim=1, keepdim=True) + 1e-6
    a1_true = G * m2 * r / r_norm**3
    a2_true = -G * m1 * r / r_norm**3

    # Mean squared ODE residual
    return ((a1 - a1_true)**2).mean() + ((a2 - a2_true)**2).mean()

def loss_fn(model, t, x1_true, x2_true):
    pred = model(t)
    x1_pred, _, x2_pred, _ = torch.split(pred, 2, dim=1)
    data_loss = nn.MSELoss()(x1_pred, x1_true) + nn.MSELoss()(x2_pred, x2_true)
    ode_loss = ode_residual(model, t, m1, m2)
    return data_loss + ode_loss, data_loss.item(), ode_loss.item()

for epoch in range(2000):
    optimizer.zero_grad()
    loss, data_loss, ode_loss = loss_fn(model, t_train, x1_train, x2_train)
    loss.backward()
    optimizer.step()
    
    if epoch % 200 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.5f}, Data Loss: {data_loss:.5f}, ODE Loss: {ode_loss:.5f}")


Epoch 0, Loss: 30.22407, Data Loss: 0.83207, ODE Loss: 29.39200
Epoch 200, Loss: 0.68464, Data Loss: 0.65963, ODE Loss: 0.02500
Epoch 400, Loss: 0.43195, Data Loss: 0.41201, ODE Loss: 0.01994
Epoch 600, Loss: 0.20039, Data Loss: 0.19320, ODE Loss: 0.00719
Epoch 800, Loss: 0.06873, Data Loss: 0.06558, ODE Loss: 0.00315
Epoch 1000, Loss: 0.03576, Data Loss: 0.03424, ODE Loss: 0.00152
Epoch 1200, Loss: 0.02640, Data Loss: 0.02540, ODE Loss: 0.00099
Epoch 1400, Loss: 0.01988, Data Loss: 0.01922, ODE Loss: 0.00066
Epoch 1600, Loss: 0.01486, Data Loss: 0.01439, ODE Loss: 0.00047
Epoch 1800, Loss: 0.01111, Data Loss: 0.01075, ODE Loss: 0.00036


In [13]:
from sklearn.metrics import mean_absolute_error

with torch.no_grad():
    t_test = torch.tensor(t_data, dtype=torch.float32)
    pred = model(t_test)
    x1_pred, _, x2_pred, _ = torch.split(pred, 2, dim=1)

    mae_x1 = mean_absolute_error(x1_true, x1_pred.numpy())
    mae_x2 = mean_absolute_error(x2_true, x2_pred.numpy())

    print(f"MAE x1: {mae_x1:.5f}, MAE x2: {mae_x2:.5f}")


MAE x1: 0.05570, MAE x2: 0.04530


In [14]:
print("Final Evaluation")
print("================")
print(f"MAE x1: {mae_x1:.5f}")
print(f"MAE x2: {mae_x2:.5f}")


Final Evaluation
MAE x1: 0.05570
MAE x2: 0.04530
